<a href="https://colab.research.google.com/github/Arun-nexus/deep_learning/blob/main/pytorch_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d iamsouravbanerjee/animal-image-dataset-90-different-animals

Dataset URL: https://www.kaggle.com/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals
License(s): other
 96% 630M/656M [00:04<00:00, 167MB/s]
100% 656M/656M [00:04<00:00, 162MB/s]


In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile("animal-image-dataset-90-different-animals.zip","r") as zip:
  zip.extractall("dataset")

In [ ]:
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,random_split
from torch import optim

In [44]:
augmentation=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

val_augmentation=transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])

In [46]:
full_data=datasets.ImageFolder("/content/dataset",transform=augmentation)
val_data=datasets.ImageFolder("/content/dataset",transform=val_augmentation)
loader=DataLoader(full_data,batch_size=64,shuffle=True,pin_memory=True,num_workers=5)

In [ ]:
from torch import nn

In [ ]:
class animal_cnn(nn.Module):
  def __init__(self,n_classes):
    super().__init__()
    self.feature=nn.Sequential(
        nn.Conv2d(3,32,3,padding=1),nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,3,padding=1),nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(64,128,3,padding=1),nn.ReLU(),
        nn.MaxPool2d(2,2),
    )
    self.dense=nn.Sequential(
        nn.Flatten(),
        nn.Linear(128*16*16,256),nn.ReLU(),
        nn.Dropout(0.25),
        nn.Linear(256,n_classes),
    )
  def forward(self,x):
    x=self.feature(x)
    x=self.dense(x)
    return x

In [50]:
import torch

In [51]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=animal_cnn(n_classes=len(full_data.classes)).to(device)
losses=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-3)


In [ ]:
class early_stopping():
  def __init__(self,patience=2,min_delta=0):
    self.patience=patience
    self.min_delta=min_delta
    self.best_loss=float("inf")
    self.counter=0
    self.early_stop=False
  def __call__(self,val_loss):
    if val_loss<self.best_loss-self.min_delta:
      self.best_loss=val_loss
      self.counter=0
    else:
      self.counter+=1
      if self.counter>=self.patience:
        self.early_stop=True

In [ ]:
epochs=3
early_stop=early_stopping()
for epoch in range(epochs):
  model.train()
  running_loss=0
  correct = 0
  total = 0
  for img,label in loader:
    img,label=img.to(device),label.to(device)
    optimizer.zero_grad()
    outputs = model(img)
    loss=losses(outputs,label)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    pred=outputs.argmax(1)
    total+=label.size(0)
    correct+=(pred == label).sum().item()

  epoch_loss=running_loss/len(loader)
  acc=100*correct/total
  print(f"epoch{epoch+1:02d}|loss{epoch_loss:.4f}|acc{acc:.2f}")

In [53]:
from PIL import  Image

In [ ]:
from google.colab import files
files.upload()

In [ ]:
def predict(img_path):
  image=Image.open(img_path).convert("RGB")
  image=transforms.Resize(1,3,128,128)
  image=transforms.ToTensor()
  model.eval()
  with torch.no_grad():
    out=model(image)
    idx = out.argmax(1).itme()
    return full_data.classes(idx)
print(predict("content/image"))
